<a href="https://colab.research.google.com/github/imusicmash/stanford_llm_python/blob/main/StanfordLLMClassWeek5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb

In [11]:
# need thes aysnc stuff later for the agent summary to work
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

# RAG

In [6]:
!pip install llama-index --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pypdf

In [2]:
!wget https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf

--2024-03-04 21:22:14--  https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 184.87.61.58
Connecting to www.goldmansachs.com (www.goldmansachs.com)|184.87.61.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491250 (480K) [application/pdf]
Saving to: ‘report.pdf’

report.pdf          100%[===================>] 479.74K  --.-KB/s    in 0.05s   

2024-03-04 21:22:15 (9.42 MB/s) - ‘report.pdf’ saved [491250/491250]



In [7]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('openai')
# client = OpenAI(api_key=open_ai_key)

In [8]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

# Routing

In [12]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("report.pdf")

# initialize settings (set chunk size)
Settings.chunk_size = 1024
# think of nodes like the chunks.
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()


Loading files:   0%|          | 0/1 [00:00<?, ?file/s]/usr/local/lib/python3.10/dist-packages/fsspec/implementations/local.py:388: RuntimeWarning: coroutine 'LLM.apredict' was never awaited
  return self.f.seek(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/fsspec/implementations/local.py:388: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return self.f.seek(*args, **kwargs)
Loading files: 100%|██████████| 1/1 [00:02<00:00,  2.63s/file]


In [30]:
# build the one central query engine, which will have several query engines
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
    #description = ""
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
    #description="Useful for generating pictures",
    #description = ""
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)


In [31]:
response = query_engine.query("What is the 2024 outlook for US GDP?")
print(response)

# note i tried to trick it by saying the vector tool description is for generating pictures.
# it properly told me it cannot answer with the context it has.

The 2024 outlook for US GDP is forecasted to be above-consensus with a growth rate of 2.1%.


In [27]:
response = query_engine.query("Summarize the document")
print(response)

The document provides an in-depth analysis and outlook on the US equity market for 2024, focusing on the S&P 500 index. It forecasts a positive year with a 5% price gain and a total return of 6%, including dividends. The report emphasizes expectations of modest economic expansion, earnings growth of 5%, and stable equity valuations. It discusses factors influencing equity appreciation, investment strategies, ownership allocations, and potential risks in the market. Additionally, the document highlights the performance and outlook of the "Magnificent 7" mega-cap tech stocks within the S&P 500 index, addressing factors affecting their future performance and associated risks. Furthermore, it provides insights into earnings forecasts, market scenarios, macroeconomic factors, and equity market trends for the upcoming year. The document also includes detailed disclosures and regulations related to investment research reports issued by Goldman Sachs in various jurisdictions worldwide, outlini

In [24]:
# we discussed exactly how it knows which to use
# someone mentined that Pydantic can also know that it's summary engine underneath.. so it may use both descriptin and type of q engine..
# someone said that if u leave off desciption it may still work
# watch pydanic video https://www.youtube.com/watch?v=yj-wSRJwrrc

# Sub Question Query Engine

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

documents = SimpleDirectoryReader("./").load_data("report.pdf")

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,
).as_query_engine()


# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="documents",
            description="Report",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

response = query_engine.query(
    "What is the outlook for the US economy?"
)

print(response)

ValueError: No files found in ..

# Calling OpenAI AssistantAPI (Code interpreter)

In [33]:
from llama_index.agent.openai import OpenAIAssistantAgent

agent = OpenAIAssistantAgent.from_new(
    name="Python agent",
    openai_tools=[{"type": "code_interpreter"}],
    instructions="You are an expert at writing python code to solve problems.",
    verbose=True
)

response = agent.chat(
    """Generate code to answer the following question:
    How much is the us population likely to grow to by 2030?
    Return and answer and the code used."""
)

In [34]:
print(str(response))

Based on the hypothetical figures provided, the estimated US population by the year 2030 is approximately 348.6 million people.

Here is the Python code used to perform the estimation:

```python
# Given data
current_population = 332_000_000  # Current US population estimate in 2023
annual_growth_rate = 0.007  # Average annual growth rate (in decimal form)
target_year = 2030
current_year = 2023
years_into_future = target_year - current_year

# Calculate the future population
future_population = current_population * ((1 + annual_growth_rate) ** years_into_future)
```

This calculation is a simplified model that assumes a constant growth rate and does not account for other demographic factors (like immigration, emigration, birth rates, and death rates) that could influence the actual future population. For a more precise prediction, a more detailed demographic model would need to be used, and updated population statistics and growth rates should be obtained from a reliable source such as

In [36]:
response = agent.chat("Calculate 2+2 and show the python code")
print(str(response))

The result of calculating 2 + 2 is 4.

Here is the Python code used to perform the calculation:

```python
# Python code to calculate 2+2
result = 2 + 2
```


In [37]:
response = agent.chat(
    """Generate code to answer the following question:
    Use the Titanic data set from Kaggle, and write python code to build a decision tree model that can predict if a passenger survived or not.
    Return and answer and the code used."""
)
print(str(response))

To build a decision tree model to predict passenger survival on the Titanic, we would need the Titanic dataset from Kaggle. Typically, this would involve downloading the data, performing data analysis, cleaning, and preprocessing, then training and testing a machine learning model.

As I don't have access to the internet to download the dataset from Kaggle, I'll assume the dataset is in the commonly used format with features like 'Pclass' (passenger class), 'Sex', 'Age', 'SibSp' (siblings/spouses aboard), 'Parch' (parents/children aboard), 'Fare', 'Embarked' (port of embarkation), and the target variable 'Survived'.

Here is a general outline of the steps you'd take to build the decision tree model using the scikit-learn library in Python:

1. Load the dataset.
2. Perform exploratory data analysis to understand the data.
3. Preprocess the data (handle missing values, convert categorical variables to numeric, etc.).
4. Split the dataset into a training set and a test set.
5. Instantiate

# ReAct

In [2]:
# more advanced.. it will choose the next best action
# seems these next few lines are about persistence in vector db
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [3]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-03-05 04:17:49--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.08s   

2024-03-05 04:17:49 (22.8 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-03-05 04:17:49--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP 

In [10]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

**********
Trace: index_construction
    |_node_parsing ->  0.758724 seconds
      |_chunking ->  0.005237 seconds
      |_chunking ->  0.000725 seconds
      |_chunking ->  0.001843 seconds
      |_chunking ->  0.000687 seconds
      |_chunking ->  0.005319 seconds
      |_chunking ->  0.001417 seconds
      |_chunking ->  0.004646 seconds
      |_chunking ->  0.004691 seconds
      |_chunking ->  0.004842 seconds
      |_chunking ->  0.004437 seconds
      |_chunking ->  0.004818 seconds
      |_chunking ->  0.004588 seconds
      |_chunking ->  0.001729 seconds
      |_chunking ->  0.000815 seconds
      |_chunking ->  0.001214 seconds
      |_chunking ->  0.000912 seconds
      |_chunking ->  0.004764 seconds
      |_chunking ->  0.004657 seconds
      |_chunking ->  0.001801 seconds
      |_chunking ->  0.005001 seconds
      |_chunking ->  0.003136 seconds
      |_chunking ->  0.005346 seconds
      |_chunking ->  0.005202 seconds
      |_chunking ->  0.003923 seconds
      |_chu

In [11]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [13]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    # context=context
)

In [18]:
# his example from class
# it's auto generating question .  it checks if it has enough information, if not, asks more questions
# reactively..  hence ReAct framework..
response = agent.chat(
    """Compare and contrast the revenue growth of Uber and Lyft in 2021, then tell me more about how the company with the highest growth did it"""
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': 'revenue growth in 2021'}
Observation: Revenue growth in 2021 was significant, with a $6.3 billion increase, representing a 57% growth compared to the previous year.
Thought: I can answer without using any more tools.
Answer: In 2021, Uber had a revenue growth of $6.3 billion, which was a 57% increase compared to the previous year. This indicates a significant growth in revenue for Uber in 2021.
**********
Trace: chat
    |_agent_step ->  3.388134 seconds
      |_llm ->  1.088354 seconds
      |_function_call ->  1.162045 seconds
        |_query ->  1.161807 seconds
          |_retrieve ->  0.219453 seconds
            |_embedding ->  0.14775 seconds
          |_synthesize ->  0.942079 seconds
            |_templating ->  6.8e-05 seconds
            |_llm ->  0.916686 seconds
      |_llm ->  1.129791 seconds
**********
In 2021, Uber had a revenue growth of $6.3 billion, which was a 57

In [19]:
response = agent.chat(
    "Compare the risk of Uber and Lyft and return a table"
)
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer: | Risk Factors      | Uber                                      | Lyft                                      |
|-------------------|-------------------------------------------|-------------------------------------------|
| COVID-19 Impact   | Significant impact on business operations | Impact on financial performance           |
| Competition       | Intense competition in the industry       | Competition in the ride-sharing market    |
| Regulatory Challenges | Legal proceedings on driver classification | Regulatory compliance and governance risks |
| Technology        | Investments in new technologies           | Autonomous vehicle technology             |
| Security Breaches | Potential security breaches               | Security breaches and data protection    |
| Market Volatility | Exposure to market volatility             | Financial performance challenges          |
| Operational Risks | Driver and rider retention, 

In [22]:
response = agent.chat(
    "Conduct an investment analysis on Lyft and Uber"
)
print(str(response))

Thought: I need to use a tool to help me conduct an investment analysis on Lyft and Uber.
Action: lyft_10k
Action Input: {'input': 'investment analysis'}
Observation: The company's investments primarily consist of cash, cash equivalents, short-term investments, and restricted investments in various financial instruments like money market funds, certificates of deposits, commercial paper, corporate bonds, and term deposits. These investments are subject to interest rate risk, but a hypothetical 100 basis points change in interest rates is not expected to have a material impact on the company's financial condition or results of operations due to the short-term nature of the investment portfolio. Additionally, the company holds long-term debt, mainly in the form of fixed-rate Convertible Senior Notes, and a hypothetical 100 basis points change in interest rates is also not anticipated to have a material impact on the company's financials.
Thought: I can answer without using any more tools

# MSFT AutoGen